In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

import sys
import pickle
import json

# Add OpenAI import
import openai

import time

# _path_libraries = os.path.join("..","libraries")

# sys.path.insert(0, _path_libraries)

# from data_lake_libraries import *
# from hf_file_management import *

import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

%load_ext autoreload
%autoreload 2

# print(containers)

In [2]:
# Set OpenAI configuration settings
openai.api_type = "azure"
openai.api_base = "https://geicohackopenai12.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = '48959db9be8b41e082b9f354db4c232f' # os.getenv("48959db9be8b41e082b9f354db4c232f")
engine = "hackgroup51gpt35turbo"

In [15]:
system_message = '''You are a helpful assistant. Summarizing the injuries in a car accident.  An example of an 
                    injury description is as follows -
                    --
                    Wife suffered injuries in the head requiring X-rays. Some injuries in the head require Stitches.
                    She alos has some bruising and airbag burn. 
                    --
                    In car accident, my right shoulder bone has been dislocated and I have been told to undergo an operation to fix it.
                    --
                    My son has some brusing in legs and arms. A few Stitches were needed in the head.
                    
                 '''
user_message = '''Can you create a car accident injury description using no more than 20 words? 
                  Please describe the injuires and medical procedures needed. Some accidents result in minor injuries
                  while others may result in medium to severe injuries. Use a tag at the beginning 
                  (Severe Injuries:, Medium Injuries:, Mild Injuries:).
                '''

In [16]:
# Build the messages array
def get_response(system_message, user_message, engine, temperature, max_tokens):
    
    messages =[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    # Call the Azure OpenAI model
    response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response

def get_list_injuries(system_message, user_message, engine, temperature, max_tokens, num_incidents, ts):
    
    data_injuries = []
    for i in range(num_incidents):
        print('Generating example: ',i)
        
        try:
            response = get_response(system_message, user_message, engine, temperature, max_tokens)
        except:
            print('Not able to GET response')
            time.sleep(ts)
        
        try:
            print('message --> ',response.choices[0].message.content)
            data_injuries.append([i,response.choices[0].message.content])
        except:
            print('Not able to EXTRACT message.')
        
    df = pd.DataFrame(data_injuries,
                      columns=['index', 'summary_injury'],
                      index=range(len(data_injuries)))
    
    return df

In [18]:
dfinjuries = get_list_injuries(system_message, user_message, 
                               engine = engine, 
                               temperature = 0.9, 
                               max_tokens = 120,
                               num_incidents = 20,
                               ts = 45
                              )
dfinjuries.head()

Generating example:  0
message -->  Severe Injuries: Multiple fractures and internal bleeding, requiring surgery and hospitalization. Long-term rehabilitation expected.
Generating example:  1
message -->  Severe injuries: Broken ribs, dislocated shoulder, head trauma requiring surgery.
Generating example:  2
message -->  Severe Injuries: Multiple fractures, internal bleeding, surgery required.
Generating example:  3
message -->  Severe Injuries: Multiple fractures in the legs and arms, head trauma requiring surgery and extended hospitalization.
Generating example:  4
message -->  Severe Injuries: Multiple fractures and internal organ damage. Surgery and hospitalization required.
Generating example:  5
message -->  Medium Injuries: Fractured rib and whiplash requiring pain management and physical therapy.
Generating example:  6
message -->  Medium Injuries: Broken arm and ribs, concussion requiring hospitalization and CT scan. Surgery needed for arm.
Generating example:  7
message -->  

,index,summary_injury
0,0,Severe Injuries: Multiple fractures and intern...
1,1,"Severe injuries: Broken ribs, dislocated shoul..."
2,2,"Severe Injuries: Multiple fractures, internal ..."
3,3,Severe Injuries: Multiple fractures in the leg...
4,4,Severe Injuries: Multiple fractures and intern...


In [20]:
fname = os.path.join('..','output', 'summary_injuries_general1.csv')
dfinjuries.to_csv(fname)

In [23]:
system_message = '''You are a helpful assistant. Summarizing the injuries in a car accident.  
                 '''
user_message = '''Can you create a car accident injury description using no more than 20 words? 
                  Please describe the injuires and medical procedures needed. Some accidents result in minor injuries
                  while others may result in medium to severe injuries. Use a tag at the beginning 
                  (Severe Injuries:, Medium Injuries:, Mild Injuries:). Focus on medium to mild injuries.
                '''

In [24]:
dfinjuries = get_list_injuries(system_message, user_message, 
                               engine = engine, 
                               temperature = 0.9, 
                               max_tokens = 120,
                               num_incidents = 20,
                               ts = 45
                              )
dfinjuries.head()

Generating example:  0
message -->  Mild to medium injuries include whiplash, cuts, bruises and broken bones requiring X-rays, stitches, and physical therapy.
Generating example:  1
message -->  Mild Injuries: Whiplash, bruises, cuts, and scrapes may require pain medication, stitches, or physical therapy for recovery.
Generating example:  2
message -->  Mild to medium injuries from car accidents may include whiplash, bruises, cuts, and minor fractures. Medical attention may include X-rays and physical therapy.
Generating example:  3
message -->  Mild to moderate injuries in car accidents may include cuts, bruises, whiplash, and sprains. Treatment may include x-rays, stitches, and physical therapy.
Generating example:  4
message -->  Mild to medium injuries from car accidents may include whiplash, cuts, bruises, and sprains. Medical treatment may include X-rays, stitches, and pain-relief medication.
Generating example:  5
message -->  Medium to mild injuries: Whiplash, cuts and bruises,

,index,summary_injury
0,0,"Mild to medium injuries include whiplash, cuts..."
1,1,"Mild Injuries: Whiplash, bruises, cuts, and sc..."
2,2,Mild to medium injuries from car accidents may...
3,3,Mild to moderate injuries in car accidents may...
4,4,Mild to medium injuries from car accidents may...


In [25]:
fname = os.path.join('..','output', 'summary_injuries_general2.csv')
dfinjuries.to_csv(fname)

In [ ]:
response = get_response(system_message, user_message, 
                        engine=engine, 
                        temperature=0.9, 
                        max_tokens=800)
print(response.choices[0].message.content)